In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [24]:
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import warnings

warnings.filterwarnings("ignore")
train_dir = '/kaggle/input/mri-dataset-for-alzheimers/alzheimer dataset/train'
test_dir = '/kaggle/input/mri-dataset-for-alzheimers/alzheimer dataset/test'
img_size = (128, 128)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    # shear_range=0.1,
    zoom_range=0.10,
    width_shift_range=0.05,
    height_shift_range=0.05,
    # horizontal_flip=True,
    rotation_range=10,
    fill_mode='nearest',
    validation_split=0.2
)

test_datagen = ImageDataGenerator(rescale=1./255)
val_test_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_generator = val_test_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='categorical',
    subset='validation',
    shuffle=False,
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=False
)

Found 3719 images belonging to 4 classes.
Found 928 images belonging to 4 classes.
Found 1813 images belonging to 4 classes.


In [28]:
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import numpy as np

classes = train_generator.classes
weights = compute_class_weight(class_weight='balanced', classes=np.unique(classes), y=classes)
class_weights = dict(zip(np.unique(classes), weights))

CNN1 = Sequential()

CNN1.add(Conv2D(64, (3,3), activation='relu', input_shape=(128,128,1)))
CNN1.add(BatchNormalization())
CNN1.add(MaxPooling2D(2,2))

CNN1.add(Conv2D(128, (3,3), activation='relu'))
CNN1.add(BatchNormalization())
CNN1.add(MaxPooling2D(2,2))

CNN1.add(Conv2D(128, (3,3), activation='relu'))
CNN1.add(BatchNormalization())
CNN1.add(MaxPooling2D(2,2))

# CNN1.add(Conv2D(128, (3,3), activation='relu', kernel_regularizer=regularizers.l2(0.001)))
# CNN1.add(BatchNormalization())
# CNN1.add(MaxPooling2D(2,2))

# CNN1.add(Conv2D(256, (3,3), activation='relu', kernel_regularizer=regularizers.l2(0.001)))
# CNN1.add(BatchNormalization())
# CNN1.add(MaxPooling2D(2,2))

CNN1.add(GlobalAveragePooling2D())

CNN1.add(Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
CNN1.add(BatchNormalization())
CNN1.add(Dropout(0.5))

# CNN1.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
# CNN1.add(BatchNormalization())
# CNN1.add(Dropout(0.2))

CNN1.add(Dense(4, activation='softmax'))

optimizer = Adam(learning_rate=3e-4)
CNN1.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
CNN1.summary()

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_29 (Conv2D)              │ (None, 126, 126, 64)   │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_38          │ (None, 126, 126, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_26 (MaxPooling2D) │ (None, 63, 63, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_30 (Conv2D)              │ (None, 61, 61, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_39          │ (None, 61, 61, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_27 (MaxPooling2D) │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_31 (Conv2D)              │ (None, 28, 28, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_40          │ (None, 28, 28, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_28 (MaxPooling2D) │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_6      │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_41          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_42          │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 249,156 (973.27 KB)

 Trainable params: 248,132 (969.27 KB)

 Non-trainable params: 1,024 (4.00 KB)

In [22]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
callbacks = [
      EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True, verbose=1),
      ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6, verbose=1),
      ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, verbose=1)
  ]

In [29]:
history = CNN1.fit(train_generator, validation_data=val_generator, class_weight=class_weights, callbacks=[callbacks], epochs=40)

Epoch 1/40
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - accuracy: 0.3420 - loss: 1.8260
Epoch 1: val_loss did not improve from 1.55338
117/117 ━━━━━━━━━━━━━━━━━━━━ 19s 113ms/step - accuracy: 0.3422 - loss: 1.8256 - val_accuracy: 0.1196 - val_loss: 1.7684 - learning_rate: 1.0000e-04
Epoch 2/40
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.4101 - loss: 1.5862
Epoch 2: val_loss did not improve from 1.55338
117/117 ━━━━━━━━━━━━━━━━━━━━ 9s 78ms/step - accuracy: 0.4100 - loss: 1.5862 - val_accuracy: 0.1196 - val_loss: 2.3157 - learning_rate: 1.0000e-04
Epoch 3/40
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.4057 - loss: 1.5835
Epoch 3: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.

Epoch 3: val_loss did not improve from 1.55338
117/117 ━━━━━━━━━━━━━━━━━━━━ 9s 79ms/step - accuracy: 0.4057 - loss: 1.5833 - val_accuracy: 0.2737 - val_loss: 2.8831 - learning_rate: 1.0000e-04
Epoch 4/40
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.4409 - loss: 1

In [ ]:
test_loss, test_acc = CNN1.evaluate(test_generator, verbose=1)
print("Test accuracy:", test_acc)